In [1]:
import os
import sys
from tqdm import tqdm
import pandas as pd

# Chemin du dossier src
chemin_src = os.path.join('.', 'src')
sys.path.append(chemin_src)


from functions import Separate_data, clean_table_mp  
from functions import read_and_check_input_values 
from functions import create_matrix_A_and_C,format_constraints_elements
from functions import format_constraints_qualite, format_constraints_MP
from functions import solve_linear_program
from functions import optimize_with_correction
from functions import  gestion_resultats,construct_result_dataframe,export_result,save_errors,remove_old_recipes

In [2]:
def create_optimal_recipe(recette,table_mp, mp_constraints, elmt_quality_constraints,dossier_data):
    
    # df_mp = mp_constraints[recette]
    # df_elmt_and_quality = elmt_quality_constraints[recette]

    # df = Separate_data(table_mp, df_mp, df_elmt_and_quality)
    # df_contraints_element, df_contraints_qualite = df[0],df[1]
    # df_mp_dispo, df_mp_indispo = df[2],df[3]
    
    df_mp, df_elmt_and_quality = mp_constraints[recette], elmt_quality_constraints[recette]
    df_contraints_element, df_contraints_qualite, df_mp_dispo, df_mp_indispo = Separate_data(table_mp, df_mp, df_elmt_and_quality)

    # Suppression des matières premières indisponibles dans
    table_mp = clean_table_mp(table_mp, df_mp_indispo)

    # Construction de la matrice A et du vecteur C
    A, C = create_matrix_A_and_C(table_mp, df_mp_dispo)
    
    # Initialisation des listes pour les contraintes
    constraints = {'A_eq': {},'b_eq': {},'A_sup': {},'b_sup': {} }

    # Mettre  les contraintes concernant les éléments
    constraints = format_constraints_elements(df_contraints_element, A,constraints)

    # Mettre  les contraintes concernant la qualité
    constraints = format_constraints_qualite(df_contraints_qualite, A,constraints)

    # Mettre les contraintes concernant les matières premières disponibles
    constraints, bounds = format_constraints_MP(df_mp_dispo, constraints)


    # Résoudre le problème d'optimisation linéaire
    method = 'simplex' #'interior-point' 'simplex'
    coefficients = [0.6, 0.65,0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
    coefficients = [0.85, 0.9, 0.95,1]
    # res, erreurs = optimize_with_correction(table_mp, C, constraints, bounds, method, coefficients)
    res, erreurs = solve_linear_program(C, constraints, bounds,method)
    
    # Gestion du resultats
    gestion_resultats(erreurs, res, df_mp_dispo, table_mp, 
                      constraints, dossier_data, recette)
    
    return 

In [3]:

if __name__ == "__main__":
    chemin_fichier = os.path.join('.', 'data', 'recipe_optimization_data.xlsm')
    # On recupere le chemin du dossier data
    dossier_data = os.path.dirname(chemin_fichier)
    # Suppression du vieux resultats
    remove_old_recipes(dossier_data)
    # Resolutions du nouveau probleme
    Recettes = [col for col in pd.read_excel(chemin_fichier, engine='openpyxl', sheet_name=2).columns if 'Unnamed' not in col] # 'calamine'
    for recette in tqdm(Recettes, desc="Processing recipes", unit="recipe"):
        table_mp, mp_constraints, elmt_quality_constraints, errors= read_and_check_input_values(chemin_fichier)
        if errors :
            save_errors(errors, dossier_data,recette)
            break;
        create_optimal_recipe(recette, table_mp, mp_constraints,elmt_quality_constraints,dossier_data)


Processing recipes:  20%|██        | 1/5 [00:00<00:02,  1.99recipe/s]

Le problème pour la recette GS 400-15 admet une solution.


Processing recipes:  40%|████      | 2/5 [00:01<00:01,  1.85recipe/s]

Le problème pour la recette GS 450-10 admet une solution.


Processing recipes:  60%|██████    | 3/5 [00:01<00:01,  1.90recipe/s]

Le problème pour la recette GS 500-7 admet une solution.


Processing recipes:  80%|████████  | 4/5 [00:02<00:00,  1.82recipe/s]

Le problème pour la recette GS 600-3 admet une solution.


Processing recipes: 100%|██████████| 5/5 [00:02<00:00,  1.88recipe/s]

Le problème pour la recette GL 250 admet une solution.


In [5]:
chemin_fichier = os.path.join('.', 'data', 'recipe_optimization_data.xlsm')
# On recupere le chemin du dossier data
dossier_data = os.path.dirname(chemin_fichier)
# Suppression du vieux resultats
remove_old_recipes(dossier_data)
# Resolutions du nouveau probleme
Recettes = [col for col in pd.read_excel(chemin_fichier, engine='openpyxl', sheet_name=2).columns if 'Unnamed' not in col] # 'calamine'
for recette in tqdm(Recettes, desc="Processing recipes", unit="recipe"):
    table_mp, mp_constraints, elmt_quality_constraints, errors= read_and_check_input_values(chemin_fichier)
    if errors :
        save_errors(errors, dossier_data,recette)
        break;
    create_optimal_recipe(recette, table_mp, mp_constraints,elmt_quality_constraints,dossier_data)
    
    break;


Processing recipes:   0%|          | 0/5 [00:00<?, ?recipe/s]

Le problème pour la recette GS 400-15 admet une solution.


In [9]:
elmt_quality_constraints['GS 400-15']

,Composant,Impurété,ONO,C,Si,Mn,Cu,Cr,P,Ni,...,Sn,Sb,Al,S,Mg,Pb,Ti,As,Bi,V
0,Valeur Min par four,1.1,0.145,3.55,1.9,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Valeur visée,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Valeur Max par four,1.22,0.20,3.65,2.1,0.2,0.1,0.05,0.03,0.02,...,0.015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
mp_constraints['GS 400-15']

,Code Article,Article,Prix,Disponible ?,Métallique ?,Part Min,Part à consommer,Part Max
0,10,Retour GL 200,460,0,1,0,NaN,1
1,11,Retour GL 250,460,0,1,0,NaN,1
2,12,Retour Jaune,510,0,1,0,NaN,1
3,13,Retour Bleu,530,1,1,0,NaN,0.35
4,14,Retour GL200 EMAIL,460,0,0,0,NaN,1
5,15,Retours GL200 CTP,450,0,0,0,NaN,1
6,92655248,ferro silicum 75 en vrac,1520,1,1,0,NaN,1
7,92658100,E3C Cubilot,430,0,1,0,NaN,1
8,FDN0151,FePh,1250,0,0,0,NaN,1
9,FDN0172,Frites Mn,507,0,0,0,NaN,1
